## Install requirements

## Import libraries

In [2]:

import numpy as np
import faiss
import logging
from sklearn.cluster import MiniBatchKMeans
import traceback
from ipywidgets import Button
import subprocess
import os
from IPython.display import clear_output
import sys
import IPython.display as ipd
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
from pyngrok import ngrok

## Preprocess data

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



current_directory = os.getcwd()
project_folder = os.path.join(current_directory, "Proyecto/project-main")
dataset_folder = os.path.join(current_directory, "Proyecto/dataset")
logs_folder = os.path.join(project_folder, "logs")
#@title 1.Preprocess Data
os.chdir(project_folder)
model_name = str(input("Inserte el nonmbre del modelo: ")) #@param {type:"string"}
#@markdown <small> Enter the path to your dataset folder (a folder with audios of the vocals you will train on), or if you want just upload the audios using the File Manager into the 'dataset' folder.
while len(os.listdir(dataset_folder)) < 1:
    print("Your dataset folder is empty.")
    sys.exit()
model_logs_folder = os.path.join(logs_folder, model_name)
os.makedirs(model_logs_folder, exist_ok=True)
with open(os.path.join(model_logs_folder, "preprocess.log"), 'w') as f:
    f.write("Starting...")
script_path = os.path.join(project_folder, "infer/modules/train/preprocess.py")
comando = [
    "python",
    script_path,
    dataset_folder,
    "40000",
    "2",
    model_logs_folder,
    "False",
    "3.0"
]
subprocess.run(comando)
with open(os.path.join(model_logs_folder, "preprocess.log"), 'r') as f:
    if 'end preprocess' in f.read():
        clear_output()
        print(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your dataset folder is correct.")

Button(button_style='success', description='✔ Success', style=ButtonStyle())


## Extract Features

In [3]:
project_folder = os.path.join(current_directory, "Proyecto/project-main")
logs_folder = os.path.join(project_folder, "logs", model_name)
os.makedirs(logs_folder, exist_ok=True)
f0method = "rmvpe_gpu" # @param ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
os.chdir(project_folder)
with open(os.path.join(logs_folder, "extract_f0_feature.log"), 'w') as f:
    f.write("Starting...")
extract_f0_script = os.path.join(project_folder, "infer/modules/train/extract/extract_f0_rmvpe.py")
extract_feature_script = os.path.join(project_folder, "infer/modules/train/extract_feature_print.py")
comando_1 = [
    "python",
    extract_f0_script,
    "1",
    "0",
    "0",
    logs_folder,
    "True"
]
comando_2 = [
    "python",
    extract_feature_script,
    "cuda:0",
    "1",
    "0",
    "0",
    logs_folder,
    "v2"
]
subprocess.run(comando_1)
subprocess.run(comando_2)

with open(os.path.join(logs_folder, "extract_f0_feature.log"), 'r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
        print(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your data was preprocessed.")

Button(button_style='success', description='✔ Success', style=ButtonStyle())


## Train Index

In [4]:

def train_index(exp_dir1, version19,n_cpu):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "¡Por favor, realiza la extracción de características primero!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "¡Por favor, realice la extracción de características primero!"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "Índice construido con éxito, added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2',6)

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))



Button(button_style='success', description='✔ Success', style=ButtonStyle())

In [5]:
save_frequency = 40
epochs = 1000
cache = True
now_dir = os.path.join(current_directory, "Proyecto", "project-main")

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "Entrenamiento finalizado. Puede revisar el registro de entrenamiento en la consola o en la carpeta experimental como train.log."

training_log = click_train(
    model_name,
    '40k',
    True,
    0,
    save_frequency,
    epochs,
    12,
    True,
    os.path.join(project_folder, "assets/pretrained_v2/f0G40k.pth"),
    os.path.join(project_folder, "assets/pretrained_v2/f0D40k.pth"),
    0,
    cache,
    True,
    'v2',
)
print(training_log)


Write filelist done
Use gpus: 0
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
INFO:DV4:{'data': {'filter_length': 2048, 'hop_length': 400, 'max_wav_value': 32768.0, 'mel_fmax': None, 'mel_fmin': 0.0, 'n_mel_channels': 125, 'sampling_rate': 40000, 'win_length': 2048, 'training_files': './logs\\DV4/filelist.txt'}, 'model': {'filter_channels': 768, 'gin_channels': 256, 'hidd

In [6]:

#@title Inference

current_directory = os.getcwd()
project_folder = os.path.join(current_directory, "Proyecto/project-main")
dataset_folder = os.path.join(current_directory, "Proyecto/dataset")
logs_folder = os.path.join(project_folder, "logs")
transpose = 0  # Modificar según tus necesidades
input_path = str(os.path.join(current_directory, "modelizar", "Voces", "Apocalipsis.mp3"))
print(input_path)
index_path = str(os.path.join(current_directory, "Proyecto", "project-main", "logs", "DV4", "trained_IVF1353_Flat_nprobe_1_DV4_v2.index"))
f0_method = "rmvpe"  # Opción de método para frecuencia fundamental
opt_path = str(os.path.join(current_directory, "Proyecto", "dataset", "cli_output.wav"))
index_rate = 0.66  # Tasa de índice
volume_normalization = 0  # Normalización de volumen
consonant_protection = 0  # Protección de consonantes
infer_cli_path = os.path.join(current_directory, "Proyecto", "project-main", "tools", "infer_cli.py")
model_path = "DV4_e1000_s32000.pth"

print(infer_cli_path)
print(opt_path)
print("python",
    infer_cli_path,
    "--f0up_key", str(transpose),
    "--input_path", '"' + input_path + '"',
    "--index_path", '"' + index_path + '"',
    "--f0method", f0_method,
    "--opt_path", '"' + opt_path + '"',
    "--model_name", model_path,
    "--index_rate", str(index_rate),
    "--device", "cuda:0",
    "--is_half", "True",
    "--filter_radius", "3",
    "--resample_sr", "0",
    "--rms_mix_rate", str(volume_normalization),
    "--protect", str(consonant_protection))

comando_ps = [
    "python",
    infer_cli_path,
    "--f0up_key", str(transpose),
    "--input_path", input_path,
    "--index_path", index_path,
    "--f0method", f0_method,
    "--opt_path", opt_path,
    "--model_name", model_path,
    "--index_rate", str(index_rate),
    "--device", "cuda:0",
    "--is_half", "True",
    "--filter_radius", "3",
    "--resample_sr", "0",
    "--rms_mix_rate", str(volume_normalization),
    "--protect", str(consonant_protection)
]

subprocess.run(["powershell", "-Command", " ".join(comando_ps)], capture_output=True, text=True)

c:\Users\Franco-SIM\Desktop\IIA\modelizar\Voces\Apocalipsis.mp3
c:\Users\Franco-SIM\Desktop\IIA\Proyecto\project-main\tools\infer_cli.py
c:\Users\Franco-SIM\Desktop\IIA\Proyecto\dataset\cli_output.wav
python c:\Users\Franco-SIM\Desktop\IIA\Proyecto\project-main\tools\infer_cli.py --f0up_key 0 --input_path "c:\Users\Franco-SIM\Desktop\IIA\modelizar\Voces\Apocalipsis.mp3" --index_path "c:\Users\Franco-SIM\Desktop\IIA\Proyecto\project-main\logs\DV4\trained_IVF1353_Flat_nprobe_1_DV4_v2.index" --f0method rmvpe --opt_path "c:\Users\Franco-SIM\Desktop\IIA\Proyecto\dataset\cli_output.wav" --model_name DV4_e1000_s32000.pth --index_rate 0.66 --device cuda:0 --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate 0 --protect 0


CompletedProcess(args=['powershell', '-Command', 'python c:\\Users\\Franco-SIM\\Desktop\\IIA\\Proyecto\\project-main\\tools\\infer_cli.py --f0up_key 0 --input_path c:\\Users\\Franco-SIM\\Desktop\\IIA\\modelizar\\Voces\\Apocalipsis.mp3 --index_path c:\\Users\\Franco-SIM\\Desktop\\IIA\\Proyecto\\project-main\\logs\\DV4\\trained_IVF1353_Flat_nprobe_1_DV4_v2.index --f0method rmvpe --opt_path c:\\Users\\Franco-SIM\\Desktop\\IIA\\Proyecto\\dataset\\cli_output.wav --model_name DV4_e1000_s32000.pth --index_rate 0.66 --device cuda:0 --is_half True --filter_radius 3 --resample_sr 0 --rms_mix_rate 0 --protect 0'], returncode=0, stdout='', stderr='2023-12-18 10:42:39 | INFO | configs.config | Found GPU NVIDIA GeForce RTX 3080\n2023-12-18 10:42:39 | INFO | infer.modules.vc.modules | Get sid: DV4_e1000_s32000.pth\n2023-12-18 10:42:39 | INFO | infer.modules.vc.modules | Loading: c:\\Users\\Franco-SIM\\Desktop\\IIA\\Proyecto\\project-main\\assets\\weights\\DV4_e1000_s32000.pth\nc:\\Users\\Franco-SIM\\